## 🏠 Study of the Effect of National Factors on Home Prices in the US

### 🎯 **Task**

Using publicly available data on national factors that influence the **supply and demand of homes in the US**, this project builds a **data science model** to analyze the effect of these variables on **home prices**.

---

### 🧠 **Approach**

The following key variables are considered for the analysis:

- **Unemployment Rate**
- **Employment Rate**
- **Per Capita GDP**
- **Median Household Income**
- **Construction Prices**
- **Consumer Price Index (CPI)**
- **Interest Rates**
- **Working Population**
- **Urban Population**
- **Housing Subsidies**
- **Number of Households**

> 📌 **Note:** As a proxy for home prices, we use the **S&P Case-Shiller Home Price Index (CSUSHPISA)**.

---

### 🔄 **Data Collection & Processing**

- Data is sourced primarily from the [Federal Reserve Economic Data (FRED)](https://fred.stlouisfed.org/).
- The data was **downloaded**, **preprocessed**, and **combined** using the **Extract-Transform-Load (ETL)** methodology.
- Since the variables had **different data frequencies**, necessary **interpolations** were applied to align all variables on a **monthly timeline**.

---

This dataset is now ready for **exploratory data analysis (EDA)** and **predictive modeling** to understand how these macroeconomic and demographic factors impact home prices in the US.


#### Importing neccessary libraries

<div style="border: 2px solid #4CAF50; background-color: #f0f9f0; padding: 16px; border-radius: 8px;">

### 📘 Importing Neccessary Libraries

</div>


In [1]:
import numpy as np
import pandas as pd

In [2]:
df_CS = pd.read_csv("CSUSHPISA.csv")

In [3]:
df_CS.head()

,observation_date,CSUSHPISA
0,1987-01-01,63.963
1,1987-02-01,64.423
2,1987-03-01,64.734
3,1987-04-01,65.130
4,1987-05-01,65.564


<div style="border: 2px solid #4CAF50; background-color: #f0f9f0; padding: 16px; border-radius: 8px;">

### 📘 Perform ETL

</div>


In [4]:
# Reading CASE-SHILLER Index into a dataframe

# Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["observation_date"])
# Selecting data till JULY 2023
mask = df_CS["DATE"] <= "2023-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.drop(["observation_date"], axis=1, inplace = True)
df_CS.tail()


Shape of the CASE-SHILLER Index:-  (439, 5)


,CSUSHPISA,DATE,Year,Month
434,298.986,2023-03-01,2023,3
435,300.249,2023-04-01,2023,4
436,302.146,2023-05-01,2023,5
437,304.028,2023-06-01,2023,6
438,306.234,2023-07-01,2023,7


In [5]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([259], inplace = True)
df_unemp.rename(columns={'observation_date': 'DATE'}, inplace=True)
print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

Unemployment Rate Data:-  (926, 2)


,DATE,UNRATE
922,2024-11-01,4.2
923,2024-12-01,4.1
924,2025-01-01,4.0
925,2025-02-01,4.1
926,2025-03-01,4.2


In [6]:
# Reading Employment Rate Data into a dataframe
df_emp = pd.read_csv("EMPRATE.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})
df_emp.drop([259], inplace = True)
df_emp.rename(columns={'observation_date': 'DATE'}, inplace=True)
print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

shape of the Employment Rate Data:-  (577, 2)


,DATE,EmpRate
573,2024-10-01,71.70908
574,2024-11-01,71.64780
575,2024-12-01,71.88726
576,2025-01-01,72.03374
577,2025-02-01,71.86045


In [7]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("GDP_per_capita.csv", names = ["DATE", "A939RX0Q048SBEA"], skiprows = 1)
df_pcgdp = df_pcgdp.rename(columns={'A939RX0Q048SBEA': 'Per_Capita_GDP'})
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

Shape of the Per Capita GDP Data:-  (312, 2)


,DATE,Per_Capita_GDP
307,2023-10-01,67858
308,2024-01-01,67981
309,2024-04-01,68320
310,2024-07-01,68696
311,2024-10-01,69006


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.


In [8]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv").drop([259])
df_Fed_rate.rename(columns={'observation_date': 'DATE'}, inplace=True)
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()

Shape of the Interest rate data:-  (848, 2)


,DATE,FEDFUNDS
844,2024-11-01,4.64
845,2024-12-01,4.48
846,2025-01-01,4.33
847,2025-02-01,4.33
848,2025-03-01,4.33


In [9]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("construction_price_ppi.csv", names = ["DATE", "WPUSI012011"], skiprows = 1)
df_cons_price_index = df_cons_price_index.rename(columns={'WPUSI012011': 'Cons_Material'})
df_cons_price_index.drop([259], inplace = True)
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

Shape of the Construction Material Data:-  (937, 2)


,DATE,Cons_Material
933,2024-10-01,325.428
934,2024-11-01,326.759
935,2024-12-01,324.957
936,2025-01-01,326.280
937,2025-02-01,329.020


In [10]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPIAUCSL"], skiprows = 1).drop([259])
df_CPI = df_CPI.rename(columns={'CPIAUCSL': 'CPI'})
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

Shape of the Consumer Price Index:-  (937, 2)


,DATE,CPI
933,2024-10-01,315.564
934,2024-11-01,316.449
935,2024-12-01,317.603
936,2025-01-01,319.086
937,2025-02-01,319.775


In [14]:
# Housing Subsidies

df_subsidy = pd.read_csv("housing_subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()


Shape of the housing subsidies:-  (64, 2)


,DATE,Subsidy
59,2019-01-01,40.185
60,2020-01-01,44.147
61,2021-01-01,45.299
62,2022-01-01,48.021
63,2023-01-01,53.573


In [15]:
# Working age population

df_working = pd.read_csv("working_age_population.csv", names = ["DATE", "LFWA64TTUSM647S"], skiprows = 1).drop([259])
df_working = df_working.rename(columns={'LFWA64TTUSM647S': 'working_age_pop'})
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

Shape of the working age population:-  (577, 2)


,DATE,working_age_pop
573,2024-10-01,209236100
574,2024-11-01,209414000
575,2024-12-01,209417300
576,2025-01-01,211570700
577,2025-02-01,211492200


In [16]:
# Real Median Household Income

df_income = pd.read_csv("median_household_income.csv", names = ["DATE", "MEHOINUSA672N"], skiprows = 1)
df_income = df_income.rename(columns={'MEHOINUSA672N': 'median_income'})
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()


Shape of the median household income data:-  (40, 2)


,DATE,median_income
35,2019-01-01,81210
36,2020-01-01,79560
37,2021-01-01,79260
38,2022-01-01,77540
39,2023-01-01,80610


In [17]:
# Total number of households

df_households = pd.read_csv("household.csv", names = ["DATE", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()


Shape of the total households data:-  (85, 2)


,DATE,Num_Households
80,2020-01-01,128451.0
81,2021-01-01,129224.0
82,2022-01-01,131202.0
83,2023-01-01,131434.0
84,2024-01-01,132216.0


In [18]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()


,CSUSHPISA,DATE,Year,Month,Per_Capita_GDP
0,63.963,1987-01-01,1987,1,37132.0
1,64.423,1987-02-01,1987,2,NaN
2,64.734,1987-03-01,1987,3,NaN
3,65.130,1987-04-01,1987,4,37454.0
4,65.564,1987-05-01,1987,5,NaN


In [20]:
df_list = [df_CS, df_working, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]

# Ensure DATE is datetime and set as index
for i in range(len(df_list)):
    df_list[i]["DATE"] = pd.to_datetime(df_list[i]["DATE"])
    df_list[i] = df_list[i].set_index("DATE")

# Concatenate with inner join to avoid NaNs from mismatched dates
df = pd.concat(df_list, axis=1, join='inner')

print(df.shape)
df.head()


(438, 10)


,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS
DATE,,,,,,,,,,
1987-01-01,63.963,1987,1,37132.0,153837100,111.4,6.6,70.15437,107.6,6.43
1987-02-01,64.423,1987,2,NaN,154050900,111.8,6.6,70.28407,107.9,6.10
1987-03-01,64.734,1987,3,NaN,154193700,112.2,6.6,70.31629,108.1,6.13
1987-04-01,65.130,1987,4,37454.0,154340400,112.7,6.3,70.51045,108.3,6.37
1987-05-01,65.564,1987,5,NaN,154526700,113.0,6.3,70.81375,108.3,6.85


In [22]:
# Merging other dataframes 
others = [df_households, df_income, df_subsidy]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy
DATE,,,,,,,,,,,,,
1987-01-01,63.963,1987,1,37132.0,153837100,111.4,6.6,70.15437,107.6,6.43,89479.0,63060,11.506
1987-02-01,64.423,1987,2,NaN,154050900,111.8,6.6,70.28407,107.9,6.10,89479.0,63060,11.506
1987-03-01,64.734,1987,3,NaN,154193700,112.2,6.6,70.31629,108.1,6.13,89479.0,63060,11.506
1987-04-01,65.130,1987,4,37454.0,154340400,112.7,6.3,70.51045,108.3,6.37,89479.0,63060,11.506
1987-05-01,65.564,1987,5,NaN,154526700,113.0,6.3,70.81375,108.3,6.85,89479.0,63060,11.506


In [23]:
print(df.shape)

(438, 13)


Check missing values (NAN)

In [24]:
df.isna().sum()

CSUSHPISA            0
Year                 0
Month                0
Per_Capita_GDP     291
working_age_pop      0
CPI                  0
UNRATE               0
EmpRate              0
Cons_Material        0
FEDFUNDS             0
Num_Households       0
median_income        0
Subsidy              0
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns are due to the unavailability of fresh data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows with missing values in the other columns. This means that we will use data from 2002 to 2022.

**Interpolation:**

Interpolation is a mathematical technique used to estimate values that are missing in a dataset based on the values of neighboring data points. It calculates intermediate values based on the existing data.



In [25]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [26]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy
DATE,,,,,,,,,,,,,
1987-01-01,63.963,1987,1,37132.000000,153837100,111.400,6.6,70.15437,107.600,6.43,89479.0,63060,11.506
1987-02-01,64.423,1987,2,37239.333333,154050900,111.800,6.6,70.28407,107.900,6.10,89479.0,63060,11.506
1987-03-01,64.734,1987,3,37346.666667,154193700,112.200,6.6,70.31629,108.100,6.13,89479.0,63060,11.506
1987-04-01,65.130,1987,4,37454.000000,154340400,112.700,6.3,70.51045,108.300,6.37,89479.0,63060,11.506
1987-05-01,65.564,1987,5,37531.000000,154526700,113.000,6.3,70.81375,108.300,6.85,89479.0,63060,11.506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-01,298.986,2023,3,66854.333333,208145900,301.643,3.5,71.85808,331.729,4.65,131434.0,80610,53.573
2023-03-01,300.249,2023,4,66945.000000,208264600,302.858,3.4,71.95604,333.366,4.83,131434.0,80610,53.573
2023-04-01,302.146,2023,5,67129.666667,208483500,303.316,3.6,71.93774,337.473,5.06,131434.0,80610,53.573


In [27]:
df.isna().sum()

CSUSHPISA          0
Year               0
Month              0
Per_Capita_GDP     0
working_age_pop    0
CPI                0
UNRATE             0
EmpRate            0
Cons_Material      0
FEDFUNDS           0
Num_Households     0
median_income      0
Subsidy            0
dtype: int64

In [28]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy
DATE,,,,,,,,,,,,,
1987-01-01,63.963,1987,1,37132.000000,153837100,111.400,6.6,70.15437,107.600,6.43,89479.0,63060,11.506
1987-02-01,64.423,1987,2,37239.333333,154050900,111.800,6.6,70.28407,107.900,6.10,89479.0,63060,11.506
1987-03-01,64.734,1987,3,37346.666667,154193700,112.200,6.6,70.31629,108.100,6.13,89479.0,63060,11.506
1987-04-01,65.130,1987,4,37454.000000,154340400,112.700,6.3,70.51045,108.300,6.37,89479.0,63060,11.506
1987-05-01,65.564,1987,5,37531.000000,154526700,113.000,6.3,70.81375,108.300,6.85,89479.0,63060,11.506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-01,298.986,2023,3,66854.333333,208145900,301.643,3.5,71.85808,331.729,4.65,131434.0,80610,53.573
2023-03-01,300.249,2023,4,66945.000000,208264600,302.858,3.4,71.95604,333.366,4.83,131434.0,80610,53.573
2023-04-01,302.146,2023,5,67129.666667,208483500,303.316,3.6,71.93774,337.473,5.06,131434.0,80610,53.573


In [29]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (438, 13)


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".


In [30]:
df.to_csv("prepared_dataset.csv")

In [31]:
us_house_price_df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
us_house_price_df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy
DATE,,,,,,,,,,,,,
1987-01-01,63.963,1987,1,37132.000000,153837100,111.4,6.6,70.15437,107.6,6.43,89479.0,63060,11.506
1987-02-01,64.423,1987,2,37239.333333,154050900,111.8,6.6,70.28407,107.9,6.10,89479.0,63060,11.506
1987-03-01,64.734,1987,3,37346.666667,154193700,112.2,6.6,70.31629,108.1,6.13,89479.0,63060,11.506
1987-04-01,65.130,1987,4,37454.000000,154340400,112.7,6.3,70.51045,108.3,6.37,89479.0,63060,11.506
1987-05-01,65.564,1987,5,37531.000000,154526700,113.0,6.3,70.81375,108.3,6.85,89479.0,63060,11.506


## To be continued...........